In [3]:
import pandas as pd

# 讀取檔案並轉換成 DataFrame
data = []
with open('../stanford-sentiment-dataset/stsa.binary.train', 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines:
        label, text = line.strip().split(' ', 1)
        data.append({'LABEL': int(label), 'TEXT': text})

df_1 = pd.DataFrame(data)

# 顯示 DataFrame
df_1

,LABEL,TEXT
0,1,"a stirring , funny and finally transporting re..."
1,0,apparently reassembled from the cutting-room f...
2,0,they presume their audience wo n't sit still f...
3,1,this is a visually stunning rumination on love...
4,1,jonathan parker 's bartleby should have been t...
...,...,...
6915,1,"painful , horrifying and oppressively tragic ,..."
6916,0,take care is nicely performed by a quintet of ...
6917,0,"the script covers huge , heavy topics in a bla..."
6918,0,a seriously bad film with seriously warped log...


In [4]:
import pandas as pd

# 讀取檔案並轉換成 DataFrame
data = []
with open('../stanford-sentiment-dataset/stsa.binary.test', 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines:
        label, text = line.strip().split(' ', 1)
        data.append({'LABEL': int(label), 'TEXT': text})

df_2 = pd.DataFrame(data)

# 顯示 DataFrame
df_2

,LABEL,TEXT
0,0,"no movement , no yuks , not much of anything ."
1,0,"a gob of drivel so sickly sweet , even the eag..."
2,0,"gangs of new york is an unapologetic mess , wh..."
3,0,"we never really feel involved with the story ,..."
4,1,this is one of polanski 's best films .
...,...,...
1816,0,"an often-deadly boring , strange reading of a ..."
1817,0,the problem with concept films is that if the ...
1818,0,"safe conduct , however ambitious and well-inte..."
1819,0,"a film made with as little wit , interest , an..."


In [6]:
combined_df = pd.concat([df_1, df_2], axis=0)
combined_df

,LABEL,TEXT
0,1,"a stirring , funny and finally transporting re..."
1,0,apparently reassembled from the cutting-room f...
2,0,they presume their audience wo n't sit still f...
3,1,this is a visually stunning rumination on love...
4,1,jonathan parker 's bartleby should have been t...
...,...,...
1816,0,"an often-deadly boring , strange reading of a ..."
1817,0,the problem with concept films is that if the ...
1818,0,"safe conduct , however ambitious and well-inte..."
1819,0,"a film made with as little wit , interest , an..."


In [24]:
import pandas as pd
raw_data = pd.read_csv('train_2022.csv')
raw_data.drop(columns=['row_id'], axis=1, inplace=True)
raw_data = pd.concat([combined_df, raw_data], axis=0)
raw_data.reset_index(drop=True, inplace=True)
raw_data

,LABEL,TEXT
0,1,"a stirring , funny and finally transporting re..."
1,0,apparently reassembled from the cutting-room f...
2,0,they presume their audience wo n't sit still f...
3,1,this is a visually stunning rumination on love...
4,1,jonathan parker 's bartleby should have been t...
...,...,...
10736,1,an imaginative comedy\/thriller .
10737,1,a savvy exploration of paranoia and insecurity...
10738,1,on the other hand for power grating you ve got...
10739,1,"like dickens with his passages , mcgrath craft..."


In [36]:
test_data = pd.read_csv('test_no_answer_2022.csv')
test_data

,row_id,TEXT
0,0,good to know if you can t find these elsewhere .
1,1,love it ! the grill plates come out and pop i...
2,2,i m convinced this was a poorly executed refur...
3,3,i would never have complained about that if it...
4,4,"the photo shows the same whole , large candie..."
...,...,...
10995,10995,i didn t quite get it the first time .
10996,10996,i ve tried installing with and without the oem...
10997,10997,i was parked at a truck stop in the cincinnati...
10998,10998,i recently bought this case after seeing some ...


# 加入情緒字典分析

In [12]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
def append_sentiment_dictionary(df):
    new_df = df.copy();
    sid = SentimentIntensityAnalyzer()
    new_df['sentiment'] = new_df['TEXT'].apply(lambda x: sid.polarity_scores(x))
    new_df = pd.concat([new_df.drop(['sentiment'], axis=1), new_df['sentiment'].apply(pd.Series)], axis=1)
    return new_df

# 資料前處理

In [13]:
 def data_lowercase(df):
    new_df = df.copy();
    new_df['TEXT'] = new_df['TEXT'].str.lower()
    return new_df

In [14]:
import contractions
def data_contraction(df):
    new_df = df.copy();
    new_df['TEXT'] = new_df['TEXT'].apply(lambda x: contractions.fix(x))
    return new_df

In [15]:
import re
def data_remove_tags_punctuations_numbers(df):
    new_df = df.copy();
    def remove_tags_punctuations_numbers(sentense):
        sentense = re.sub(r'<[^>]+>', '', sentense)
        sentense = re.sub(r'[^\w\s]', '', sentense)
        sentense = re.sub(r'\d+', '', sentense)
        return sentense;
    new_df['TEXT'] = new_df['TEXT'].apply(lambda x: remove_tags_punctuations_numbers(x))
    return new_df

In [16]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def data_remove_stopword(df):
    new_df = df.copy();
    def remove_stopword(sentence):
        sentence_arr = sentence.split()
        filtered_sentence = [word for word in sentence_arr if word.lower() not in stop_words]
        filtered_sentence = ' '.join(filtered_sentence)
        return filtered_sentence
    new_df['TEXT'] = new_df['TEXT'].apply(lambda x: remove_stopword(x))
    return new_df

In [17]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
lemmatizer = WordNetLemmatizer()

def data_lemmatize_text(df):
    new_df = df.copy();
    def lemmatize_text(text):
        tokens = word_tokenize(text)
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
        return ' '.join(lemmatized_tokens)
    new_df['TEXT'] = new_df['TEXT'].apply(lambda x: lemmatize_text(x))
    return new_df

In [18]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer(language='english')

def data_stem(df):
    new_df = df.copy();
    new_df['TEXT'] = new_df['TEXT'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
    return new_df

In [19]:
def data_preprocess(df):
    df = data_lowercase(df)
    df = data_contraction(df)
    df = data_remove_tags_punctuations_numbers(df)
    df = data_remove_stopword(df)
    df = data_lemmatize_text(df)
    df = data_stem(df)
    return df

# 加入TF-IDF

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

def append_tfidf(df):
    new_df = df.copy()
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(new_df['TEXT'])
    X_tfidf_df = pd.DataFrame(list(tfidf_matrix.toarray()), columns=['tfidf_'+str(i) for i in range(tfidf_matrix.shape[1])])
    new_df = pd.concat([new_df, X_tfidf_df], axis=1)
    return new_df

# 情緒字典 + SVM

In [21]:
%%time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

train_data = raw_data.copy()
train_data = append_sentiment_dictionary(train_data)

X = train_data.drop(columns=['TEXT','LABEL'])

y = train_data['LABEL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = SVC(kernel='linear')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.6919497440670079
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.71      0.69      1020
           1       0.72      0.67      0.70      1129

    accuracy                           0.69      2149
   macro avg       0.69      0.69      0.69      2149
weighted avg       0.69      0.69      0.69      2149

CPU times: total: 1.2 s
Wall time: 3.55 s


# TF-IDF + 情緒字典 + SVM

In [13]:
%%time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

train_data = raw_data.copy()
train_data = append_sentiment_dictionary(train_data)
train_data = append_tfidf(train_data)

X = train_data.drop(columns=['TEXT','LABEL'])

y = train_data['LABEL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = SVC(kernel='linear')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.635
Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.46      0.54       189
           1       0.62      0.80      0.70       211

    accuracy                           0.64       400
   macro avg       0.64      0.63      0.62       400
weighted avg       0.64      0.64      0.62       400

CPU times: total: 3min 31s
Wall time: 9min 17s


# 資料前處理 + TF-IDF + 情緒字典 + SVM

In [25]:
train_data = raw_data.copy()
train_data = append_sentiment_dictionary(train_data)
train_data = data_preprocess(train_data)
train_data = append_tfidf(train_data)
train_data

,LABEL,TEXT,neg,neu,pos,compound,tfidf_0,tfidf_1,tfidf_2,tfidf_3,...,tfidf_11975,tfidf_11976,tfidf_11977,tfidf_11978,tfidf_11979,tfidf_11980,tfidf_11981,tfidf_11982,tfidf_11983,tfidf_11984
0,1,stir funni final transport reimagin beauti bea...,0.165,0.536,0.299,0.4588,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,appar reassembl cuttingroom floor given daytim...,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,presum audienc wo nt sit still sociolog lesson...,0.000,0.912,0.088,0.4404,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,visual stun rumin love memori histori war art ...,0.165,0.549,0.287,0.4404,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,jonathan parker bartlebi beallendal modernoffi...,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10736,1,imagin comedythril,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10737,1,savvi explor paranoia insecur america cultur fear,0.447,0.447,0.106,-0.7269,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10738,1,hand power grate got someth,0.096,0.685,0.219,0.4215,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10739,1,like dicken passag mcgrath craft quit move sce...,0.000,0.769,0.231,0.5859,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
%%time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

train_data = raw_data.copy()
train_data = append_sentiment_dictionary(train_data)
train_data = data_preprocess(train_data)
train_data = append_tfidf(train_data)

X = train_data.drop(columns=['TEXT','LABEL'])

y = train_data['LABEL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = SVC(kernel='linear')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.8124709167054444
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.84      0.81      1020
           1       0.85      0.79      0.82      1129

    accuracy                           0.81      2149
   macro avg       0.81      0.81      0.81      2149
weighted avg       0.81      0.81      0.81      2149

CPU times: total: 2min 15s
Wall time: 23min 28s


# 測試集
### 全1: 應該差不多
### 全0: 0.50055

In [37]:
test = test_data.copy()
test = append_sentiment_dictionary(test)
test = data_preprocess(test)
test = append_tfidf(test)
test = test.drop(columns=['TEXT','row_id'])
test

,neg,neu,pos,compound,tfidf_0,tfidf_1,tfidf_2,tfidf_3,tfidf_4,tfidf_5,...,tfidf_6661,tfidf_6662,tfidf_6663,tfidf_6664,tfidf_6665,tfidf_6666,tfidf_6667,tfidf_6668,tfidf_6669,tfidf_6670
0,0.000,0.734,0.266,0.4404,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000,0.616,0.384,0.8439,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,0.847,0.153,0.4019,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000,0.861,0.139,0.3089,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10996,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10997,0.196,0.804,0.000,-0.2960,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10998,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
# 檢查測試資料的特徵名稱和順序
print(test.columns)

# 確認特徵名稱和順序與訓練資料相同
expected_columns = ['TEXT', 'neg', 'neu', 'pos', 'compound'] + [f'tfidf_{i}' for i in range(11985)]
if list(test.columns) != expected_columns:
    # 調整特徵名稱和順序
    test = test[expected_columns]
    print("調整後的測試資料特徵名稱和順序：", test.columns)
else:
    print("測試資料特徵名稱和順序正確。")

# 使用模型進行預測
y_pred_test = model.predict(test.drop(columns=['TEXT']))
# 將預測結果與 row_id 合併成新的 DataFrame
predictions = pd.DataFrame({'row_id': test_data['row_id'], 'LABEL': y_pred_test})

# 顯示預測結果
print(predictions)

Index(['neg', 'neu', 'pos', 'compound', 'tfidf_0', 'tfidf_1', 'tfidf_2',
       'tfidf_3', 'tfidf_4', 'tfidf_5',
       ...
       'tfidf_6661', 'tfidf_6662', 'tfidf_6663', 'tfidf_6664', 'tfidf_6665',
       'tfidf_6666', 'tfidf_6667', 'tfidf_6668', 'tfidf_6669', 'tfidf_6670'],
      dtype='object', length=6675)


KeyError: "['TEXT', 'tfidf_6671', 'tfidf_6672', 'tfidf_6673', 'tfidf_6674', 'tfidf_6675', 'tfidf_6676', 'tfidf_6677', 'tfidf_6678', 'tfidf_6679', 'tfidf_6680', 'tfidf_6681', 'tfidf_6682', 'tfidf_6683', 'tfidf_6684', 'tfidf_6685', 'tfidf_6686', 'tfidf_6687', 'tfidf_6688', 'tfidf_6689', 'tfidf_6690', 'tfidf_6691', 'tfidf_6692', 'tfidf_6693', 'tfidf_6694', 'tfidf_6695', 'tfidf_6696', 'tfidf_6697', 'tfidf_6698', 'tfidf_6699', 'tfidf_6700', 'tfidf_6701', 'tfidf_6702', 'tfidf_6703', 'tfidf_6704', 'tfidf_6705', 'tfidf_6706', 'tfidf_6707', 'tfidf_6708', 'tfidf_6709', 'tfidf_6710', 'tfidf_6711', 'tfidf_6712', 'tfidf_6713', 'tfidf_6714', 'tfidf_6715', 'tfidf_6716', 'tfidf_6717', 'tfidf_6718', 'tfidf_6719', 'tfidf_6720', 'tfidf_6721', 'tfidf_6722', 'tfidf_6723', 'tfidf_6724', 'tfidf_6725', 'tfidf_6726', 'tfidf_6727', 'tfidf_6728', 'tfidf_6729', 'tfidf_6730', 'tfidf_6731', 'tfidf_6732', 'tfidf_6733', 'tfidf_6734', 'tfidf_6735', 'tfidf_6736', 'tfidf_6737', 'tfidf_6738', 'tfidf_6739', 'tfidf_6740', 

In [9]:
predict_data = test_data.copy()
predict_data['LABEL'] = 0
predict_data = predict_data.drop(columns=['TEXT'])
predict_data

,row_id,LABEL
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
10995,10995,0
10996,10996,0
10997,10997,0
10998,10998,0


In [5]:
import datetime
import pytz
def export_csv(df,name):
  now = datetime.datetime.now().astimezone(pytz.timezone('Asia/Taipei'))
  formatted_time = now.strftime('%Y-%m-%d %H時%M分%S秒')
  df.to_csv('predict_data/'+ name + '_' + formatted_time + ".csv", index=False,encoding="utf_8_sig")

In [10]:
export_csv(predict_data,'all_zero')